In [1]:
!pip install google-cloud-storage


In [1]:
import pandas as pd 

In [4]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.6 MB/s eta 0:00:00


In [7]:
import pandas as pd

# Install fastparquet or pyarrow to enable .to_parquet
# !pip install fastparquet

# Define the path to the output Parquet file
chunksize = 10000
reader = pd.read_stata("gs://my-dataproc-staging-bucket/Data_input/Newnis2021_merged.dta", chunksize=chunksize)

for i, chunk in enumerate(reader):
    # Example transform
    # chunk["new_column"] = chunk["existing_column"] * 2
    
    # Write each chunk to a separate file
    chunk.to_parquet(f"transformed_data_v1_{i}.parquet", index=False, engine="fastparquet")

print("Transformed data saved in chunked Parquet files.")

Transformed data saved in chunked Parquet files.


In [ ]:
import pandas as pd

# Make sure fastparquet is installed:
# !pip install fastparquet

output_parquet_path = "transformed_data_v1.parquet"
chunksize = 10000

reader = pd.read_stata("gs://my-dataproc-staging-bucket/Data_input/Newnis2021_merged.dta", chunksize=chunksize)

for i, chunk in enumerate(reader):
    # Example transformation
    # chunk["new_column"] = chunk["existing_column"] * 2
    
    # Write the first chunk
    if i == 0:
        chunk.to_parquet(output_parquet_path, engine="fastparquet", index=False)
    else:
        # Append subsequent chunks into the same parquet dataset
        chunk.to_parquet(output_parquet_path, engine="fastparquet", index=False, append=True)

print(f"All chunks appended into parquet dataset: {output_parquet_path}")

In [2]:
# Define the path to the output CSV file
output_csv_path = "transformed_data_v1.csv"

# Read the .dta file in chunks
chunksize = 10000  # Adjust the chunk size based on your memory capacity
reader = pd.read_stata("gs://my-dataproc-staging-bucket/Data_input/Newnis2021_merged.dta", chunksize=chunksize)

# Process and save each chunk
for i, chunk in enumerate(reader):
    # Apply your transformations here
    # Example: chunk['new_column'] = chunk['existing_column'] * 2
    
    # Save the transformed chunk to CSV
    if i == 0:
        chunk.to_csv(output_csv_path, index=False, mode='w', header=True)
    else:
        chunk.to_csv(output_csv_path, index=False, mode='a', header=False)

print(f"Transformed data saved to {output_csv_path}")

Transformed data saved to transformed_data_v1.csv


In [7]:
!pip install optuna

In [8]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.0 MB/s eta 0:00:00


In [10]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 76.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 MB 69.8 MB/s eta 0:00:0000:0100:01


## Importing Required Packages 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from lightgbm import LGBMClassifier
import optuna
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.preprocessing import KBinsDiscretizer

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [1]:
import pandas as pd 
df = pd.read_csv("gs://my-dataproc-staging-bucket/Data_ETL/transformed_data_v1.csv")
print(df.head(5))

/var/tmp/ipykernel_23526/4117799187.py:2: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("gs://my-dataproc-staging-bucket/Data_ETL/transformed_data_v1.csv")


   HOSP_NIS  YEAR  CCR_NIS  WAGEINDEX    DISCWT  HOSP_BEDSIZE  HOSP_DIVISION  \
0     50385  2021    0.327      0.887  5.000010             3              5   
1     50378  2021    0.196      0.948  4.999997             3              5   
2     30454  2021    0.252      0.940  5.000006             3              3   
3     50568  2021    0.440      1.124  4.999997             3              5   
4     70434  2021    0.100      0.929  5.000000             1              7   

   HOSP_LOCTEACH  HOSP_REGION  H_CONTRL  ...  new  fy2021finalpostacutedrg  \
0              3            3         1  ...  1.0                       No   
1              3            3         2  ...  2.0                       No   
2              3            2         2  ...  3.0                       No   
3              3            3         2  ...  4.0                       No   
4              3            3         3  ...  5.0                       No   

   fy2021finalspecialpaydrg  mdc  type  \
0       

## Understanding the data 

In [15]:
df.shape

(6666752, 75)

In [16]:
df.columns

Index(['HOSP_NIS', 'YEAR', 'CCR_NIS', 'WAGEINDEX', 'DISCWT', 'HOSP_BEDSIZE',
       'HOSP_DIVISION', 'HOSP_LOCTEACH', 'HOSP_REGION', 'H_CONTRL',
       'NIS_STRATUM', 'N_DISC_U', 'N_HOSP_U', 'S_DISC_U', 'S_HOSP_U',
       'TOTAL_DISC', 'AGE', 'AMONTH', 'AWEEKEND', 'DIED', 'DISPUNIFORM',
       'DQTR', 'DRG', 'DRGVER', 'DRG_NoPOA', 'ELECTIVE', 'FEMALE', 'HCUP_ED',
       'I10_BIRTH', 'I10_DELIVERY', 'KEY_NIS', 'LOS', 'MDC', 'MDC_NoPOA',
       'PAY1', 'PCLASS_ORPROC', 'PL_NCHS', 'RACE', 'TOTCHG', 'TRAN_IN',
       'TRAN_OUT', 'ZIPINC_QRTL', 'CMR_AIDS', 'CMR_ALCOHOL', 'CMR_AUTOIMMUNE',
       'CMR_CANCER_LYMPH', 'CMR_CANCER_LEUK', 'CMR_CANCER_METS',
       'CMR_CANCER_NSITU', 'CMR_CANCER_SOLID', 'CMR_DEMENTIA', 'CMR_DEPRESS',
       'CMR_DIAB_UNCX', 'CMR_DIAB_CX', 'CMR_DRUG_ABUSE', 'CMR_HTN_CX',
       'CMR_HTN_UNCX', 'CMR_LUNG_CHRONIC', 'CMR_OBESE', 'CMR_PERIVASC',
       'CMR_THYROID_HYPO', 'CMR_THYROID_OTH', 'APRDRG',
       'APRDRG_Risk_Mortality', 'APRDRG_Severity', 'new',
       'f

In [17]:
list(df.columns)

['HOSP_NIS',
 'YEAR',
 'CCR_NIS',
 'WAGEINDEX',
 'DISCWT',
 'HOSP_BEDSIZE',
 'HOSP_DIVISION',
 'HOSP_LOCTEACH',
 'HOSP_REGION',
 'H_CONTRL',
 'NIS_STRATUM',
 'N_DISC_U',
 'N_HOSP_U',
 'S_DISC_U',
 'S_HOSP_U',
 'TOTAL_DISC',
 'AGE',
 'AMONTH',
 'AWEEKEND',
 'DIED',
 'DISPUNIFORM',
 'DQTR',
 'DRG',
 'DRGVER',
 'DRG_NoPOA',
 'ELECTIVE',
 'FEMALE',
 'HCUP_ED',
 'I10_BIRTH',
 'I10_DELIVERY',
 'KEY_NIS',
 'LOS',
 'MDC',
 'MDC_NoPOA',
 'PAY1',
 'PCLASS_ORPROC',
 'PL_NCHS',
 'RACE',
 'TOTCHG',
 'TRAN_IN',
 'TRAN_OUT',
 'ZIPINC_QRTL',
 'CMR_AIDS',
 'CMR_ALCOHOL',
 'CMR_AUTOIMMUNE',
 'CMR_CANCER_LYMPH',
 'CMR_CANCER_LEUK',
 'CMR_CANCER_METS',
 'CMR_CANCER_NSITU',
 'CMR_CANCER_SOLID',
 'CMR_DEMENTIA',
 'CMR_DEPRESS',
 'CMR_DIAB_UNCX',
 'CMR_DIAB_CX',
 'CMR_DRUG_ABUSE',
 'CMR_HTN_CX',
 'CMR_HTN_UNCX',
 'CMR_LUNG_CHRONIC',
 'CMR_OBESE',
 'CMR_PERIVASC',
 'CMR_THYROID_HYPO',
 'CMR_THYROID_OTH',
 'APRDRG',
 'APRDRG_Risk_Mortality',
 'APRDRG_Severity',
 'new',
 'fy2021finalpostacutedrg',
 'fy2021final

1. Patient and Demographic Information
Variable Description
AGE Patient&#39;s age in years.
FEMALE Indicator for gender: 0 = Male, 1 = Female.
RACE Patient&#39;s race/ethnicity.
ZIPINC_QRT
L

Median household income national quartile for patient ZIP
Code.

2. Admission and Discharge Information
Variable

Description

ELECTIVE Indicator for elective admissions: 0 = Non-elective, 1 =

Elective.

AMONTH Month of admission (1 = January, 12 = December).
AWEEKEND Weekend admission indicator: 0 = Weekday, 1 = Weekend.
DISPUNIFOR
M

Uniform patient discharge status (e.g., discharged home,
died).

DQTR Quarter of discharge (1 = Jan-Mar, 2 = Apr-Jun, etc.).
DIED In-hospital mortality indicator: 0 = No, 1 = Yes.

3. Hospital Characteristics
Variable Description
HOSP_NIS Unique identifier for hospitals.
H_CONTRL Hospital ownership/control (e.g., public, private non-profit, etc.).
HOSP_LOCTEA Hospital location and teaching status (e.g., urban, rural, teaching).

CH
HOSP_REGION U.S. Census region of the hospital (Northeast, Midwest, South,

West).

4. Clinical and Severity Indicators
Variable Description
CMR_AIDS Comorbidity: AIDS/HIV.
CMR_ALCOHOL Comorbidity: Alcohol abuse.
CMR_OBESE Comorbidity: Obesity.
CMR_CANCER_LYM
PH Comorbidity: Lymphoma.
CMR_CANCER_LEU
K Comorbidity: Leukemia.
CMR_CANCER_MET
S Comorbidity: Metastatic cancer.
CMR_CANCER_NSIT
U Comorbidity: In situ cancer.
CMR_CANCER_SOLI
D Comorbidity: Solid tumors.
CMR_DIAB_UNCX Comorbidity: Uncomplicated

diabetes.

CMR_DIAB_CX Comorbidity: Complicated diabetes.
CMR_LUNG_CHRONI
C Comorbidity: Chronic lung disease.
CMR_DRUG_ABUSE Comorbidity: Drug abuse.

5. Financial and Resource Use
Variable Description

TOTCHG Total hospital charges for the patient stay.
LOS Length of hospital stay in days.
TOTAL_DIS
C Total number of discharges for a hospital in a year.
PAY1 Primary expected payer (e.g., Medicare, Medicaid, private

insurance).

6. Procedures and Diagnoses
Variable Description
MDC Major Diagnostic Category based on principal

diagnosis.

7. Survey Design Variables
Variable Description
NIS_STRATU
M

Stratum for stratified sampling (e.g., region, bed
size).

HOSP_NIS Primary sampling unit (PSU).
DISCWT Discharge weight for survey analysis.

In [18]:
df.head()

,HOSP_NIS,YEAR,CCR_NIS,WAGEINDEX,DISCWT,HOSP_BEDSIZE,HOSP_DIVISION,HOSP_LOCTEACH,HOSP_REGION,H_CONTRL,NIS_STRATUM,N_DISC_U,N_HOSP_U,S_DISC_U,S_HOSP_U,TOTAL_DISC,AGE,AMONTH,AWEEKEND,DIED,DISPUNIFORM,DQTR,DRG,DRGVER,DRG_NoPOA,ELECTIVE,FEMALE,HCUP_ED,I10_BIRTH,I10_DELIVERY,KEY_NIS,LOS,MDC,MDC_NoPOA,PAY1,PCLASS_ORPROC,PL_NCHS,RACE,TOTCHG,TRAN_IN,TRAN_OUT,ZIPINC_QRTL,CMR_AIDS,CMR_ALCOHOL,CMR_AUTOIMMUNE,CMR_CANCER_LYMPH,CMR_CANCER_LEUK,CMR_CANCER_METS,CMR_CANCER_NSITU,CMR_CANCER_SOLID,CMR_DEMENTIA,CMR_DEPRESS,CMR_DIAB_UNCX,CMR_DIAB_CX,CMR_DRUG_ABUSE,CMR_HTN_CX,CMR_HTN_UNCX,CMR_LUNG_CHRONIC,CMR_OBESE,CMR_PERIVASC,CMR_THYROID_HYPO,CMR_THYROID_OTH,APRDRG,APRDRG_Risk_Mortality,APRDRG_Severity,new,fy2021finalpostacutedrg,fy2021finalspecialpaydrg,mdc,type,msdrgtitle,weights,geometricmeanlos,arithmeticmeanlos,_merge
0,50385,2021,0.327,0.887,5.000010,3,5,3,3,1,5133,499691,15,99938,15,7983,20.0,1.0,0.0,0.0,6.0,1.0,1,38,1,1.0,0.0,0,0,0,51238854.0,33.0,5,5,3.0,1,3.0,1.0,1277233.0,0.0,0.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,161,3,3,1.0,No,No,PRE,SURG,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,28.9652,30.2,39.2,Matched (3)
1,50378,2021,0.196,0.948,4.999997,3,5,3,3,2,5233,1906579,58,381316,58,8321,62.0,1.0,0.0,0.0,6.0,1.0,1,38,1,0.0,0.0,1,0,0,50416582.0,26.0,5,5,1.0,1,2.0,1.0,1161213.0,0.0,0.0,3.0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,161,4,4,2.0,No,No,PRE,SURG,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,28.9652,30.2,39.2,Matched (3)
2,30454,2021,0.252,0.940,5.000006,3,3,3,2,2,3233,1714962,63,342992,63,5008,50.0,8.0,0.0,0.0,6.0,3.0,1,38,1,0.0,0.0,0,0,0,30229860.0,24.0,5,5,6.0,1,1.0,1.0,1160309.0,1.0,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,161,4,4,3.0,No,No,PRE,SURG,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,28.9652,30.2,39.2,Matched (3)
3,50568,2021,0.440,1.124,4.999997,3,5,3,3,2,5233,1906579,58,381316,58,11895,55.0,7.0,0.0,0.0,5.0,3.0,1,38,1,0.0,0.0,0,0,0,50307613.0,57.0,5,5,1.0,1,2.0,2.0,1086597.0,0.0,2.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,161,4,4,4.0,No,No,PRE,SURG,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,28.9652,30.2,39.2,Matched (3)
4,70434,2021,0.100,0.929,5.000000,1,7,3,3,3,7331,255875,55,51175,54,2541,67.0,6.0,0.0,0.0,6.0,2.0,1,38,1,0.0,0.0,0,0,0,70629442.0,6.0,5,5,1.0,1,1.0,1.0,154412.0,1.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,182,4,3,5.0,No,No,PRE,SURG,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,28.9652,30.2,39.2,Matched (3)


In [20]:
df.value_counts

<bound method DataFrame.value_counts of          HOSP_NIS  YEAR  CCR_NIS  WAGEINDEX    DISCWT  HOSP_BEDSIZE  \
0           50385  2021    0.327      0.887  5.000010             3   
1           50378  2021    0.196      0.948  4.999997             3   
2           30454  2021    0.252      0.940  5.000006             3   
3           50568  2021    0.440      1.124  4.999997             3   
4           70434  2021    0.100      0.929  5.000000             1   
...           ...   ...      ...        ...       ...           ...   
6666747     80033  2021    0.095      0.984  5.000000             3   
6666748     10104  2021    0.311      1.061  4.999973             1   
6666749     10131  2021    0.321      1.106  4.999973             1   
6666750     40008  2021    0.730      0.792  5.000030             3   
6666751     10126  2021    0.335      1.057  5.000000             3   

         HOSP_DIVISION  HOSP_LOCTEACH  HOSP_REGION  H_CONTRL  NIS_STRATUM  \
0                    5        

In [21]:
df.describe()

,HOSP_NIS,YEAR,CCR_NIS,WAGEINDEX,DISCWT,HOSP_BEDSIZE,HOSP_DIVISION,HOSP_LOCTEACH,HOSP_REGION,H_CONTRL,NIS_STRATUM,N_DISC_U,N_HOSP_U,S_DISC_U,S_HOSP_U,TOTAL_DISC,AGE,AMONTH,AWEEKEND,DIED,DISPUNIFORM,DQTR,DRG,DRGVER,DRG_NoPOA,ELECTIVE,FEMALE,HCUP_ED,I10_BIRTH,I10_DELIVERY,KEY_NIS,LOS,MDC,MDC_NoPOA,PAY1,PCLASS_ORPROC,PL_NCHS,RACE,TOTCHG,TRAN_IN,TRAN_OUT,ZIPINC_QRTL,CMR_AIDS,CMR_ALCOHOL,CMR_AUTOIMMUNE,CMR_CANCER_LYMPH,CMR_CANCER_LEUK,CMR_CANCER_METS,CMR_CANCER_NSITU,CMR_CANCER_SOLID,CMR_DEMENTIA,CMR_DEPRESS,CMR_DIAB_UNCX,CMR_DIAB_CX,CMR_DRUG_ABUSE,CMR_HTN_CX,CMR_HTN_UNCX,CMR_LUNG_CHRONIC,CMR_OBESE,CMR_PERIVASC,CMR_THYROID_HYPO,CMR_THYROID_OTH,APRDRG,APRDRG_Risk_Mortality,APRDRG_Severity,new,weights,geometricmeanlos,arithmeticmeanlos
count,6.666752e+06,6666752.0,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666498e+06,6.657676e+06,6.666712e+06,6.665003e+06,6.665003e+06,6.657700e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.656852e+06,6.664205e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666456e+06,6.666752e+06,6.666752e+06,6.655903e+06,6.666752e+06,6.619940e+06,6.425116e+06,6.609677e+06,6.630247e+06,6.665003e+06,6.556740e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.666752e+06,6.657704e+06,6.657704e+06,6.657704e+06
mean,5.119220e+04,2021.0,2.858437e-01,1.003658e+00,4.999994e+00,2.267366e+00,5.089098e+00,2.665834e+00,2.619817e+00,2.020414e+00,5.320065e+03,6.793447e+05,4.514577e+01,1.358689e+05,4.417809e+01,3.970749e+03,4.980889e+01,6.555386e+00,2.162109e-01,3.054372e-02,2.918602e+00,2.518407e+00,5.483249e+02,3.825062e+01,5.480581e+02,1.718922e-01,5.509852e-01,7.726367e-01,1.048919e-01,1.046315e-01,5.133816e+07,5.155402e+00,1.018708e+01,1.019325e+01,2.125737e+00,2.481276e-01,2.631188e+00,1.725052e+00,6.711614e+04,1.421944e-01,2.770672e-01,2.350856e+00,5.428731e-03,4.963571e-02,2.782779e-02,8.499341e-03,6.738514e-03,2.763955e-02,2.656466e-04,2.498473e-02,5.393991e-02,1.122736e-01,8.367193e-02,1.610832e-01,4.919082e-02,1.960985e-01,2.642179e-01,1.780311e-01,1.763784e-01,5.201678e-02,1.100615e-01,1.172010e-02,4.115363e+02,1.969444e+00,2.211047e+00,3.333376e+06,1.671115e+00,4.003142e+00,4.995762e+00
std,2.428850e+04,0.0,1.526233e-01,2.070512e-01,1.096575e-04,8.067719e-01,2.428210e+00,6.233396e-01,9.967430e-01,4.985109e-01,2.430438e+03,5.866015e+05,3.511242e+01,1.173203e+05,3.501472e+01,2.786154e+03,2.720978e+01,3.422618e+00,4.116598e-01,1.720779e-01,3.844152e+00,1.110210e+00,2.835402e+02,4.333722e-01,2.834275e+02,3.772868e-01,4.973938e-01,7.487458e-01,3.075520e-01,3.060780e-01,2.427883e+07,7.898899e+00,5.836937e+00,5.834311e+00,1.177374e+00,4.319263e-01,1.522217e+00,1.199248e+00,1.350763e+05,4.419918e-01,6.766738e-01,1.114642e+00,7.347966e-02,2.171912e-01,1.644792e-01,9.179926e-02,8.181141e-02,1.639378e-01,1.629650e-02,1.560785e-01,2.258991e-01,3.157027e-01,2.768952e-01,3.676077e-01,2.162662e-01,3.970439e-01,4.409159e-01,3.825389e-01,3.811418e-01,2.220609e-01,3.129665e-01,1.076231e-01,2.427379e+02,1.092370e+00,1.003208e+00,1.924526e+06,1.647283e+00,2.340523e+00,2.989400e+00
min,1.000100e+04,2021.0,4.200000e-02,6.840000e-01,4.993548e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.211000e+03,7.740000e+02,2.000000e+00,1.550000e+02,2.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+07,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+02,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0

In [25]:
df['LOS'].dtypes

dtype('float64')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6666752 entries, 0 to 6666751
Data columns (total 75 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   HOSP_NIS                  int64  
 1   YEAR                      int64  
 2   CCR_NIS                   float64
 3   WAGEINDEX                 float64
 4   DISCWT                    float64
 5   HOSP_BEDSIZE              int64  
 6   HOSP_DIVISION             int64  
 7   HOSP_LOCTEACH             int64  
 8   HOSP_REGION               int64  
 9   H_CONTRL                  int64  
 10  NIS_STRATUM               int64  
 11  N_DISC_U                  int64  
 12  N_HOSP_U                  int64  
 13  S_DISC_U                  int64  
 14  S_HOSP_U                  int64  
 15  TOTAL_DISC                int64  
 16  AGE                       float64
 17  AMONTH                    float64
 18  AWEEKEND                  float64
 19  DIED                      float64
 20  DISPUNIFORM             

In [35]:
df.isnull().sum()

HOSP_NIS                0
YEAR                    0
CCR_NIS                 0
WAGEINDEX               0
DISCWT                  0
                     ... 
msdrgtitle              0
weights              9048
geometricmeanlos     9048
arithmeticmeanlos    9048
_merge                  0
Length: 75, dtype: int64

In [38]:
df['CMR_OBESE']

0          0
1          0
2          0
3          0
4          1
          ..
6666747    0
6666748    0
6666749    0
6666750    0
6666751    0
Name: CMR_OBESE, Length: 6666752, dtype: int64

In [39]:
df['CMR_DIAB_CX']

0          0
1          1
2          1
3          0
4          0
          ..
6666747    0
6666748    1
6666749    1
6666750    0
6666751    0
Name: CMR_DIAB_CX, Length: 6666752, dtype: int64

In [40]:
df['CMR_DIAB_UNCX']

0          0
1          0
2          0
3          0
4          1
          ..
6666747    0
6666748    0
6666749    0
6666750    0
6666751    0
Name: CMR_DIAB_UNCX, Length: 6666752, dtype: int64

## Exploratory Data Analysis and Taking out Important Columns

In [2]:
# 2. Feature Selection
features = [
    # Demographics
    'AGE', 'FEMALE', 'RACE', 'ZIPINC_QRT',
    # Admission
    'ELECTIVE', 'AWEEKEND', 'DIED',
    # Hospital
    'H_CONTRL', 'HOSP_LOCTEA', 'HOSP_REGION',
    # Clinical
    'CMR_AIDS', 'CMR_ALCOHOL', 'CMR_OBESE', 'CMR_DIAB_UNCX', 
    'CMR_DIAB_CX', 'CMR_LUNG_CHRONIC', 'CMR_DRUG_ABUSE',
   'CMR_AUTOIMMUNE', 'CMR_CANCER_LYMPH', 'CMR_CANCER_LEUK','CMR_CANCER_METS',
   'CMR_CANCER_NSITU', 'CMR_CANCER_SOLID', 'CMR_DEMENTIA', 'CMR_DEPRESS', 'CMR_HTN_CX', 'CMR_HTN_UNCX',
    'CMR_LUNG_CHRONIC', 'CMR_PERIVASC', 'CMR_THYROID_HYPO', 'CMR_THYROID_OTH'
]
target = 'LOS'

In [55]:
 df[['ELECTIVE', 'AWEEKEND', 'DIED']]

,ELECTIVE,AWEEKEND,DIED
0,1.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
6666747,0.0,1.0,0.0
6666748,0.0,0.0,1.0
6666749,0.0,0.0,1.0
6666750,0.0,0.0,0.0


In [3]:
# Selected only columns to be used for the Model development 

selected_columns = ['HOSP_NIS', 'YEAR',
    'AGE', 'FEMALE', 'RACE',
    'ELECTIVE', 'AWEEKEND', 'DIED',
    'H_CONTRL', 'HOSP_REGION',
    'CMR_AIDS', 'CMR_ALCOHOL', 'CMR_OBESE', 'CMR_DIAB_UNCX', 
    'CMR_DIAB_CX', 'CMR_LUNG_CHRONIC', 'CMR_DRUG_ABUSE',
   'CMR_AUTOIMMUNE', 'CMR_CANCER_LYMPH', 'CMR_CANCER_LEUK',
    'CMR_CANCER_METS', 'CMR_CANCER_NSITU', 'CMR_CANCER_SOLID', 
    'CMR_DEMENTIA', 'CMR_DEPRESS', 'CMR_HTN_CX', 'CMR_HTN_UNCX',
    'CMR_LUNG_CHRONIC', 'CMR_PERIVASC', 'CMR_THYROID_HYPO',
    'CMR_THYROID_OTH', 'LOS'
]


In [4]:
df[selected_columns]

,HOSP_NIS,YEAR,AGE,FEMALE,RACE,ELECTIVE,AWEEKEND,DIED,H_CONTRL,HOSP_REGION,...,CMR_CANCER_SOLID,CMR_DEMENTIA,CMR_DEPRESS,CMR_HTN_CX,CMR_HTN_UNCX,CMR_LUNG_CHRONIC,CMR_PERIVASC,CMR_THYROID_HYPO,CMR_THYROID_OTH,LOS
0,50385,2021,20.0,0.0,1.0,1.0,0.0,0.0,1,3,...,0,0,0,0,0,0,0,0,0,33.0
1,50378,2021,62.0,0.0,1.0,0.0,0.0,0.0,2,3,...,0,0,1,0,0,0,0,1,0,26.0
2,30454,2021,50.0,0.0,1.0,0.0,0.0,0.0,2,2,...,0,0,0,0,0,1,0,0,0,24.0
3,50568,2021,55.0,0.0,2.0,0.0,0.0,0.0,2,3,...,0,0,0,0,0,0,0,0,0,57.0
4,70434,2021,67.0,0.0,1.0,0.0,0.0,0.0,3,3,...,0,0,0,1,0,1,0,0,0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6666747,80033,2021,40.0,0.0,1.0,0.0,1.0,0.0,3,4,...,0,0,0,0,0,0,0,0,0,9.0
6666748,10104,2021,63.0,0.0,1.0,0.0,0.0,1.0,2,1,...,0,0,0,1,0,0,0,0,0,12.0
6666749,10131,2021,75.0,1.0,1.0,0.0,0.0,1.0,2,1,...,0,0,0,1,0,0,0,0,0,4.0
6666750,40008,2021,25.0,0.0,6.0,0.0,0.0,0.0,2,2,...,0,0,0,0,0,0,0,0,0,5.0


In [6]:
df_model = df[selected_columns]

df_model.to_csv('hospital_model_data.csv')

In [60]:
print("Available columns")
print(df.columns.tolist())

Available columns
['HOSP_NIS', 'YEAR', 'CCR_NIS', 'WAGEINDEX', 'DISCWT', 'HOSP_BEDSIZE', 'HOSP_DIVISION', 'HOSP_LOCTEACH', 'HOSP_REGION', 'H_CONTRL', 'NIS_STRATUM', 'N_DISC_U', 'N_HOSP_U', 'S_DISC_U', 'S_HOSP_U', 'TOTAL_DISC', 'AGE', 'AMONTH', 'AWEEKEND', 'DIED', 'DISPUNIFORM', 'DQTR', 'DRG', 'DRGVER', 'DRG_NoPOA', 'ELECTIVE', 'FEMALE', 'HCUP_ED', 'I10_BIRTH', 'I10_DELIVERY', 'KEY_NIS', 'LOS', 'MDC', 'MDC_NoPOA', 'PAY1', 'PCLASS_ORPROC', 'PL_NCHS', 'RACE', 'TOTCHG', 'TRAN_IN', 'TRAN_OUT', 'ZIPINC_QRTL', 'CMR_AIDS', 'CMR_ALCOHOL', 'CMR_AUTOIMMUNE', 'CMR_CANCER_LYMPH', 'CMR_CANCER_LEUK', 'CMR_CANCER_METS', 'CMR_CANCER_NSITU', 'CMR_CANCER_SOLID', 'CMR_DEMENTIA', 'CMR_DEPRESS', 'CMR_DIAB_UNCX', 'CMR_DIAB_CX', 'CMR_DRUG_ABUSE', 'CMR_HTN_CX', 'CMR_HTN_UNCX', 'CMR_LUNG_CHRONIC', 'CMR_OBESE', 'CMR_PERIVASC', 'CMR_THYROID_HYPO', 'CMR_THYROID_OTH', 'APRDRG', 'APRDRG_Risk_Mortality', 'APRDRG_Severity', 'new', 'fy2021finalpostacutedrg', 'fy2021finalspecialpaydrg', 'mdc', 'type', 'msdrgtitle', 'wei

In [7]:
df_model.dtypes

HOSP_NIS              int64
YEAR                  int64
AGE                 float64
FEMALE              float64
RACE                float64
ELECTIVE            float64
AWEEKEND            float64
DIED                float64
H_CONTRL              int64
HOSP_REGION           int64
CMR_AIDS              int64
CMR_ALCOHOL           int64
CMR_OBESE             int64
CMR_DIAB_UNCX         int64
CMR_DIAB_CX           int64
CMR_LUNG_CHRONIC      int64
CMR_DRUG_ABUSE        int64
CMR_AUTOIMMUNE        int64
CMR_CANCER_LYMPH      int64
CMR_CANCER_LEUK       int64
CMR_CANCER_METS       int64
CMR_CANCER_NSITU      int64
CMR_CANCER_SOLID      int64
CMR_DEMENTIA          int64
CMR_DEPRESS           int64
CMR_HTN_CX            int64
CMR_HTN_UNCX          int64
CMR_LUNG_CHRONIC      int64
CMR_PERIVASC          int64
CMR_THYROID_HYPO      int64
CMR_THYROID_OTH       int64
LOS                 float64
dtype: object

In [8]:
df_model.rename(columns={'FEMALE': 'GENDER'}, inplace=True)

/var/tmp/ipykernel_23526/3607152845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.rename(columns={'FEMALE': 'GENDER'}, inplace=True)


In [9]:
# lets's find why LOS is a float, if we need to change it to an int64

df_model['LOS'].sum() # to see if there is a value after the .

34368260.0

In [11]:
# also lets try this to confirm 
df_model['LOS'].sample(n=20)

4336460     4.0
3193363    19.0
272105      2.0
5861749     5.0
2104759     7.0
826352      2.0
3294530     1.0
4325642     1.0
4757083     2.0
5941624     1.0
1413328    11.0
1811652     6.0
3386650     2.0
508596      2.0
6508830     2.0
4241239     7.0
3614092     6.0
939442      5.0
5202190     2.0
1367327     1.0
Name: LOS, dtype: float64

In [12]:
df_model.dtypes

HOSP_NIS              int64
YEAR                  int64
AGE                 float64
GENDER              float64
RACE                float64
ELECTIVE            float64
AWEEKEND            float64
DIED                float64
H_CONTRL              int64
HOSP_REGION           int64
CMR_AIDS              int64
CMR_ALCOHOL           int64
CMR_OBESE             int64
CMR_DIAB_UNCX         int64
CMR_DIAB_CX           int64
CMR_LUNG_CHRONIC      int64
CMR_DRUG_ABUSE        int64
CMR_AUTOIMMUNE        int64
CMR_CANCER_LYMPH      int64
CMR_CANCER_LEUK       int64
CMR_CANCER_METS       int64
CMR_CANCER_NSITU      int64
CMR_CANCER_SOLID      int64
CMR_DEMENTIA          int64
CMR_DEPRESS           int64
CMR_HTN_CX            int64
CMR_HTN_UNCX          int64
CMR_LUNG_CHRONIC      int64
CMR_PERIVASC          int64
CMR_THYROID_HYPO      int64
CMR_THYROID_OTH       int64
LOS                 float64
dtype: object

In [ ]:
import pandas as pd
pd.read_csv('Hospital_occupancy/hospital_model_data.csv')

In [ ]:
df_model.AGE.isnull().sum()

In [23]:
df_model.dropna(subset=['AGE'], inplace=True)
df['column_name'] = df['column_name'].astype(int) 

/var/tmp/ipykernel_23526/1149716446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['AGE'], inplace=True)


KeyError: 'column_name'

In [24]:
# convert all floats back to int64. som are categorical data that has been change to 1s and 0s but they come out as a float which might be a problem for our model

df_model['AGE']    =   df_model['AGE'].astype('int')        


/var/tmp/ipykernel_23526/3454951154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['AGE']    =   df_model['AGE'].astype('int')


In [26]:
 df_model.dropna(subset=['GENDER'], inplace=True)     
df_model['GENDER'] =  df_model['GENDER'].astype('int') 


/var/tmp/ipykernel_23526/1133989151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['GENDER'], inplace=True)
/var/tmp/ipykernel_23526/1133989151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['GENDER'] =  df_model['GENDER'].astype('int')


In [27]:
df_model.dropna(subset=['RACE'], inplace=True) 
df_model['RACE']  =  df_model['RACE'].astype('int') 
 

/var/tmp/ipykernel_23526/2033263336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['RACE'], inplace=True)
/var/tmp/ipykernel_23526/2033263336.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['RACE']  =  df_model['RACE'].astype('int')


In [29]:
df_model.dropna(subset=['ELECTIVE'], inplace=True) 
df_model['ELECTIVE'] = df_model['ELECTIVE'].astype('int') 


/var/tmp/ipykernel_23526/1889885909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['ELECTIVE'], inplace=True)
/var/tmp/ipykernel_23526/1889885909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['ELECTIVE'] = df_model['ELECTIVE'].astype('int')


In [30]:
df_model.dropna(subset=['AWEEKEND'], inplace=True) 
df_model['AWEEKEND'] = df_model['AWEEKEND'].astype('int') 


/var/tmp/ipykernel_23526/2664192348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['AWEEKEND'], inplace=True)
/var/tmp/ipykernel_23526/2664192348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['AWEEKEND'] = df_model['AWEEKEND'].astype('int')


In [31]:
df_model.dropna(subset=['DIED'], inplace=True) 
df_model['DIED']  = df_model['DIED'].astype('int') 
 

/var/tmp/ipykernel_23526/250673449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['DIED'], inplace=True)
/var/tmp/ipykernel_23526/250673449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['DIED']  = df_model['DIED'].astype('int')


In [32]:
df_model.dropna(subset=['LOS'], inplace=True) 
df_model['LOS'] = df_model['LOS'].astype('int')


/var/tmp/ipykernel_23526/268025527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.dropna(subset=['LOS'], inplace=True)
/var/tmp/ipykernel_23526/268025527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['LOS'] = df_model['LOS'].astype('int')


In [33]:
df_model.dtypes

HOSP_NIS            int64
YEAR                int64
AGE                 int64
GENDER              int64
RACE                int64
ELECTIVE            int64
AWEEKEND            int64
DIED                int64
H_CONTRL            int64
HOSP_REGION         int64
CMR_AIDS            int64
CMR_ALCOHOL         int64
CMR_OBESE           int64
CMR_DIAB_UNCX       int64
CMR_DIAB_CX         int64
CMR_LUNG_CHRONIC    int64
CMR_DRUG_ABUSE      int64
CMR_AUTOIMMUNE      int64
CMR_CANCER_LYMPH    int64
CMR_CANCER_LEUK     int64
CMR_CANCER_METS     int64
CMR_CANCER_NSITU    int64
CMR_CANCER_SOLID    int64
CMR_DEMENTIA        int64
CMR_DEPRESS         int64
CMR_HTN_CX          int64
CMR_HTN_UNCX        int64
CMR_LUNG_CHRONIC    int64
CMR_PERIVASC        int64
CMR_THYROID_HYPO    int64
CMR_THYROID_OTH     int64
LOS                 int64
dtype: object

In [34]:
df_model.shape

(6411237, 32)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
# First split: 80% train+validation, 20% test
X = df.drop('LOS', axis=1)  # Features
y = df['LOS']               # Target

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Second split: 75% train, 25% validation (from remaining 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

# Final split ratios:
# Train: 60% (0.8 * 0.75)
# Validation: 20% (0.8 * 0.25) 
# Test: 20%